## Here we reduce and concatenate the stats

In [3]:
import pandas as pd
import numpy as np
import os
import json
from tqdm import tqdm

def get_fixed_column_names():
    """
    Generate a fixed list of column names to ensure consistency across all files
    """
    columns = []
    
    # Models (excluding model_5)
    models = [f"model_{i}" for i in range(1, 7) if i != 5]
    
    # Metrics for each model
    metrics = ['gold_lp', 'rank', 'Entropy', 'gold_lp_cumsum', 'gold_prob', 'gold_lp_normalized']
    
    # Statistics to compute
    stats = ['median', 'max', 'min', 'std', 
             'moment1', 'moment2', 'moment3', 'moment4', 'moment5', 
             'q95', 'q90', 'q85', 'q80', 'q20', 'q15', 'q10', 'q05']
    
    # Generate columns for model metrics
    for model in models:
        for metric in metrics:
            for stat in stats:
                columns.append(f"{model}_{metric}_{stat}")
    
    # Generate columns for KL divergence terms
    model_pairs = [(i, j) for i in range(1, 7) for j in range(i+1, 7) 
                  if i != 5 and j != 5]
    
    for i, j in model_pairs:
        for stat in stats:
            columns.append(f"kl_{i}_vs_{j}_{stat}")
    
    return columns

def get_label_from_json(trial_name, question_number, label_folder_path):
    """
    Get label from corresponding JSON file
    """
    if not label_folder_path:
        return None
        
    json_path = os.path.join(label_folder_path, f"{trial_name}.json")
    
    try:
        with open(json_path, 'r') as f:
            data = json.load(f)
            statement = data.get('label')
            return statement
    except Exception as e:
        print(f"Error reading JSON for {trial_name}: {e}")
        return None

def extract_trial_and_question(file_path):
    """
    Extract trial name and question number from file path
    """
    try:
        filename = os.path.basename(file_path)
        question_num = int(filename.split('_')[-1].split('.')[0])
        trial_name = file_path.split('/')[-2]
        return trial_name, question_num
    except Exception as e:
        print(f"Error extracting trial and question from {file_path}: {e}")
        return None, None

def compute_statistics(series):
    """
    Compute various statistical measures for a series
    """
    mean = series.mean()
    
    stats = {
        'median': series.median(),
        'max': series.max(),
        'min': series.min(),
        'std': series.std(),
        
        # Central moments
        'moment1': mean,
        'moment2': ((series - mean) ** 2).mean(),
        'moment3': ((series - mean) ** 3).mean(),
        'moment4': ((series - mean) ** 4).mean(),
        'moment5': ((series - mean) ** 5).mean(),
        
        # Existing quantiles
        'q95': series.quantile(0.95),
        'q90': series.quantile(0.90),
        'q85': series.quantile(0.85),
        'q80': series.quantile(0.80),
        'q20': series.quantile(0.20),
        'q15': series.quantile(0.15),
        'q10': series.quantile(0.10),
        'q05': series.quantile(0.05),
    }    
    return stats

def process_metrics_and_kl(input_folder, kl_folder, output_folder, label, label_folder_path=None):
    """
    Process original metrics and KL divergence terms with consistent column ordering
    """
    # Get fixed column names
    fixed_columns = get_fixed_column_names()
    
    # Original metrics to analyze
    metrics = ['gold_lp', 'rank', 'Entropy', 'gold_lp_cumsum', 'gold_prob', 'gold_lp_normalized']
    
    # Get all CSV files from model1 directory
    csv_files = []
    model1_path = os.path.join(input_folder, "model_1")
    for root, dirs, files in os.walk(model1_path):
        # Skip hidden directories
        dirs[:] = [d for d in dirs if not d.startswith('.')]
        
        for file in files:
            if not file.startswith('.') and file.endswith('.csv'):
                csv_files.append((os.path.join(root, file), file))
    
    print(f"Found {len(csv_files)} CSV files to process")
    print(f"Output will have {len(fixed_columns) + 1} columns")  # +1 for label
    
    for file_path1, file_name in tqdm(csv_files, desc="Processing files"):
        try:
            # Get trial name and question number
            trial_name, question_num = extract_trial_and_question(file_path1)
            
            if trial_name and question_num and label_folder_path:
                # Get label from JSON only if path is provided
                label = get_label_from_json(trial_name, question_num, label_folder_path)
            else:
                label = None
            
            # Dictionary to store all metrics
            all_stats = {col: np.nan for col in fixed_columns}  # Initialize with NaN
            
            # Add label column
            all_stats['label'] = label   # fact or hall
             
            # Process each model (excluding model_5)
            models = [f"model_{i}" for i in range(1, 7) if i != 5]
            
            # Step 1: Process original metrics for each model
            for model in models:
                file_path = os.path.join(input_folder, model, os.path.relpath(file_path1, model1_path))
                if not os.path.exists(file_path):
                    print(f"Skipping {file_name} - no matching file in {model}")
                    continue
                
                df = pd.read_csv(file_path)
                
                # Compute statistics for each metric
                for metric in metrics:
                    if metric in df.columns:
                        stats = compute_statistics(df[metric])
                        for stat_name, value in stats.items():
                            col_name = f"{model}_{metric}_{stat_name}"
                            all_stats[col_name] = value
            
            # Step 2: Process KL divergence terms
            kl_file_path = os.path.join(kl_folder, os.path.relpath(file_path1, model1_path))
            if os.path.exists(kl_file_path):
                kl_df = pd.read_csv(kl_file_path)
                
                # Get all KL columns
                kl_cols = [col for col in kl_df.columns if col.startswith('kl_')]
                
                # Compute statistics for each KL term
                for kl_col in kl_cols:
                    stats = compute_statistics(kl_df[kl_col])
                    for stat_name, value in stats.items():
                        col_name = f"{kl_col}_{stat_name}"
                        all_stats[col_name] = value
            
            # Create output DataFrame with fixed column order plus label
            columns_with_label = fixed_columns + ['label']
            result_df = pd.DataFrame([all_stats])[columns_with_label]
            
            # Create output directory structure
            output_file_path = os.path.join(output_folder, os.path.relpath(file_path1, model1_path))
            os.makedirs(os.path.dirname(output_file_path), exist_ok=True)
            
            # Save results
            result_df.to_csv(output_file_path, index=False)
            
        except Exception as e:
            print(f"\nError processing {file_name}:")
            print(f"Error type: {type(e)}")
            print(f"Error message: {str(e)}")
            continue

def concatenate_output_files(output_folder):
    """
    Concatenate all CSV files in Output_folder and its subfolders into a single DataFrame
    """
    # Get list of all CSV files
    csv_files = []
    for root, dirs, files in os.walk(output_folder):
        # Skip hidden directories
        dirs[:] = [d for d in dirs if not d.startswith('.')]
        
        for file in files:
            if not file.startswith('.') and file.endswith('.csv'):
                csv_files.append((os.path.join(root, file), file))
    
    print(f"Found {len(csv_files)} CSV files to concatenate")
    # Sort files alphabetically
    csv_files = sorted(csv_files)
    
    # Read and concatenate all files
    all_dfs = []
    for file_path, file_name in tqdm(csv_files, desc="Reading files"):
        try:
            df = pd.read_csv(file_path)
            
            # Add filename as a column (optional but useful for tracking)
            df['source_file'] = file_name
            
            # Add full path as a column (optional)
            df['file_path'] = os.path.relpath(file_path, output_folder)
            
            all_dfs.append(df)
            
        except Exception as e:
            print(f"\nError reading {file_name}:")
            print(f"Error type: {type(e)}")
            print(f"Error message: {str(e)}")
            continue
    
    # Concatenate all DataFrames
    if all_dfs:
        final_df = pd.concat(all_dfs, ignore_index=True)
        
        # Print some information about the final DataFrame
        print("\nFinal DataFrame info:")
        print(f"Shape: {final_df.shape}")
        print(f"Number of samples: {len(final_df)}")
        print(f"Number of features: {len(final_df.columns)}")
        
        if 'label' in final_df.columns:
            print("\nLabel distribution:")
            print(final_df['label'].value_counts())
        
        return final_df
    else:
        print("No files were successfully read!")
        return None

if __name__ == "__main__":
    # Paths
    input_folder = "Database_free_evaluation/Clinical_note_summarization/FEATURES/Paragraph_fact"
    kl_folder = "kl_analysis_Paragraph_fact"
    output_folder = "Output_folder_Paragraph_fact"

     # Label folder path (set to None if not using labels)
    label = "fact" # or hallucination
    label_folder_path = "Database_free_evaluation/Clinical_note_summarization/Data"
    # Or for other folders:
    # label_folder_path = None
    
    # Create output directory if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Step 1: Process files and create individual outputs
    process_metrics_and_kl(input_folder, kl_folder, output_folder, label, label_folder_path)

    # Step 2: Concatenate all output files
    final_df = concatenate_output_files(output_folder)
    
    # Optional: Save concatenated DataFrame
    if final_df is not None:
        final_df.to_csv("concatenated_results_Paragraph_hallucination_CliSumm.csv", index=False)

Found 2617 CSV files to process
Output will have 681 columns


Processing files: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2617/2617 [04:40<00:00,  9.32it/s]


Found 2617 CSV files to concatenate


Reading files: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2617/2617 [00:20<00:00, 129.45it/s]



Final DataFrame info:
Shape: (2617, 683)
Number of samples: 2617
Number of features: 683

Label distribution:
label
hallucination    2617
Name: count, dtype: int64


In [4]:
final_df.head(20)

,model_1_gold_lp_median,model_1_gold_lp_max,model_1_gold_lp_min,model_1_gold_lp_std,model_1_gold_lp_moment1,model_1_gold_lp_moment2,model_1_gold_lp_moment3,model_1_gold_lp_moment4,model_1_gold_lp_moment5,model_1_gold_lp_q95,...,kl_4_vs_6_q90,kl_4_vs_6_q85,kl_4_vs_6_q80,kl_4_vs_6_q20,kl_4_vs_6_q15,kl_4_vs_6_q10,kl_4_vs_6_q05,label,source_file,file_path
0,-0.822691,-1.311301e-06,-10.000000,4.177192,-2.923535,15.510163,-66.575946,591.884490,-3.844082e+03,-1.835821e-06,...,3.127899,0.725249,0.595459,0.002655,9.342272e-04,7.176728e-05,6.764674e-05,hallucination,logprob_matrix_1.csv,NCT00000/logprob_matrix_1.csv
1,-0.150930,0.000000e+00,-16.895998,5.296153,-3.588167,26.399286,-171.560395,2342.924760,-2.712908e+04,0.000000e+00,...,0.726329,0.517785,0.460546,0.000446,1.191447e-04,1.355378e-05,2.362285e-06,hallucination,logprob_matrix_1.csv,NCT00001/logprob_matrix_1.csv
2,-1.070499,-7.152555e-07,-10.565603,4.384261,-3.315674,17.940297,-67.790062,639.098578,-3.823883e+03,-1.299380e-06,...,0.883796,0.718479,0.699422,0.000434,1.032866e-04,2.841335e-05,9.079434e-06,hallucination,logprob_matrix_1.csv,NCT00002/logprob_matrix_1.csv
3,-0.055905,0.000000e+00,-10.662206,3.815994,-2.347643,13.868394,-68.651390,599.515743,-4.476176e+03,0.000000e+00,...,0.561278,0.479652,0.298720,0.000658,3.575428e-04,8.115973e-05,5.588063e-05,hallucination,logprob_matrix_1.csv,NCT00003/logprob_matrix_1.csv
4,-3.956551,-1.617539e-04,-10.000000,4.595864,-4.395010,18.104541,-15.951117,446.473826,-8.906206e+02,-2.282903e-04,...,4.649381,1.638078,0.576365,0.014100,1.102521e-02,1.021643e-02,9.407650e-03,hallucination,logprob_matrix_1.csv,NCT00004/logprob_matrix_1.csv
5,-1.827200,0.000000e+00,-10.000000,4.452884,-3.816397,18.411876,-43.784902,517.154856,-2.239405e+03,-2.324581e-07,...,0.760579,0.437889,0.402184,0.010118,7.383154e-03,2.331821e-03,2.891298e-07,hallucination,logprob_matrix_1.csv,NCT00005/logprob_matrix_1.csv
6,-0.089566,0.000000e+00,-16.449772,4.194138,-2.682800,16.939281,-121.857068,1597.634410,-1.991269e+04,0.000000e+00,...,0.828362,0.508419,0.228717,0.000008,2.544614e-07,5.814401e-08,5.395472e-09,hallucination,logprob_matrix_1.csv,NCT00006/logprob_matrix_1.csv
7,-0.113322,0.000000e+00,-18.861296,4.898445,-3.268756,23.106071,-171.934512,2624.356636,-3.670493e+04,0.000000e+00,...,0.347967,0.314251,0.264991,0.000393,1.818667e-05,5.267619e-06,8.044870e-07,hallucination,logprob_matrix_1.csv,NCT00007/logprob_matrix_1.csv
8,-0.001308,0.000000e+00,-13.975663,4.799787,-2.789749,20.943597,-148.217905,1705.237451,-1.759254e+04,0.000000e+00,...,1.839701,1.318755,0.797810,0.123694,6.191552e-02,1.368674e-04,7.680264e-05,hallucination,logprob_matrix_1.csv,NCT00008/logprob_matrix_1.csv
9,-0.262987,-7.033324e-06,-10.000000,4.752104,-3.073193,19.356422,-78.506330,706.350118,-4.411267e+03,-1.425070e-04,...,3.730877,1.503496,0.730652,0.003832,1.370186e-03,1.280827e-03,1.191467e-03,hallucination,logprob_matrix_1.csv,NCT00009/logprob_matrix_1.csv
